In [1]:
import pandas as pd
import os
import numpy as np
import sqlite3

In [2]:
path = os.path.join('data', 'За периода 01.01.2025 г. - 31.03.2025 г..csv')
df = pd.read_csv(path, encoding='utf-8-sig', engine='python', quotechar='"')
df.shape

(100021, 17)

In [3]:
df.columns

Index(['SETTLEMENT_DATE', 'CLIENT_RECEIVER_NAME', 'CLIENT_RECEIVER_ACC',
       'CLIENT_RECEIVER_BIC', 'FIN_CODE', 'FIN_NAME', 'AMOUNT', 'CURRENCY',
       'REASON1', 'REASON2', 'REG_DATE', 'REG_NO', 'SEBRA_PAY_CODE',
       'ORGANIZATION', 'PRIMARY_ORGANIZATION', 'PRIMARY_ORG_CODE',
       'CLIENT_NAME_HASH'],
      dtype='object')

In [4]:
date_cols = ['SETTLEMENT_DATE', 'REG_DATE']
for col in date_cols:
    df[col] = pd.to_datetime(df[col], format='%d.%m.%Y')

In [5]:
df = df.drop('CURRENCY', axis='columns')
df['ORGANIZATION'] = df['ORGANIZATION'].str.replace('Централно управление', 'ЦУ')
for x in df.query('ORGANIZATION == "ЦУ"')['PRIMARY_ORGANIZATION'].unique():
    df.loc[(df['PRIMARY_ORGANIZATION']==x) & (df['ORGANIZATION'] == 'ЦУ'), 'ORGANIZATION'] = x+' - ЦУ'

In [6]:
assert df.groupby('PRIMARY_ORG_CODE')['PRIMARY_ORGANIZATION'].nunique().nunique() == 1
assert df.groupby('ORGANIZATION')['PRIMARY_ORG_CODE'].nunique().nunique() == 1
orgs = df.groupby('ORGANIZATION', as_index=False)[['PRIMARY_ORGANIZATION', 'PRIMARY_ORG_CODE']].first()
assert orgs['ORGANIZATION'].nunique() == len(orgs)

In [7]:
df['ORGANIZATION_ID'] = df['ORGANIZATION'].map(orgs['ORGANIZATION'].reset_index().rename({'index': 'ORGANIZATION_ID'}, axis=1).set_index('ORGANIZATION')['ORGANIZATION_ID'])
df = df.drop(['ORGANIZATION', 'PRIMARY_ORGANIZATION', 'PRIMARY_ORG_CODE'], axis='columns')

In [8]:
assert orgs.groupby('PRIMARY_ORG_CODE')['PRIMARY_ORGANIZATION'].nunique().nunique() == 1
primary_orgs = orgs.groupby('PRIMARY_ORG_CODE')['PRIMARY_ORGANIZATION'].first()
orgs = orgs.drop('PRIMARY_ORGANIZATION', axis='columns')

In [9]:
assert df.groupby('CLIENT_NAME_HASH')['CLIENT_RECEIVER_NAME'].nunique().nunique() == 1
clients = df.groupby('CLIENT_NAME_HASH', as_index=False)['CLIENT_RECEIVER_NAME'].first()

In [10]:
df['CLIENT_ID'] = df['CLIENT_NAME_HASH'].map(clients['CLIENT_NAME_HASH'].reset_index().rename({'index': 'CLIENT_ID'}, axis=1).set_index('CLIENT_NAME_HASH')['CLIENT_ID'])
df = df.drop(['CLIENT_NAME_HASH', 'CLIENT_RECEIVER_NAME'], axis=1)
clients = clients['CLIENT_RECEIVER_NAME']

In [11]:
df = df[['SETTLEMENT_DATE', 'AMOUNT', 'SEBRA_PAY_CODE', 'ORGANIZATION_ID', 'CLIENT_ID', 'REASON1', 'REASON2']]

In [12]:
sebra = pd.Series(
    data={
        1: 'Заплати, възнаграждения и други плащания за персонала - нетна сума за изплащане',
        2: 'Удържани данъци и осигурителни вноски от заплати и възнаграждения за персонала',
        3: 'Плащания за други удръжки от възнаграждения за персонала',
        5: 'Осигурителни вноски за сметка на осигурителя (с изключение на централизирано разплащане на осигурителни вноски)',
        10: 'Издръжка',
        18: 'Други разходи',
        20: 'Разходи за лихви',
        30: 'Текущи субсидии за предприятия',
        40: 'Стипендии, пенсии, помощи и текущи трансфери за домакинства',
        50: 'Плащания за дълготрайни активи, основен ремонт и капиталови трансфери',
        60: 'Трансфери за бюджетни и извънбюджетни сметки',
        70: 'Възмездно финансиране - придобиване на дялове и акции, предоставени кредити и временна финансова помощ',
        80: 'Погашения по банкови заеми',
        88: 'Средства на разпореждане - плащания по операции с временно съхранявани средства и средства на разпореждане',
        89: 'Друго финансиране - плащания за сметка на ЕС, плащания по операции с чужди средства и всички други операции в частта на финансирането, които не са  обхванати от кодове 70, 80 и 88.',
        90: 'Възстановени приходи - плащания за възстановяване на надвнесени и погрешно внесени бюджетни приходи',
        91: 'Теглене на левове в брой',
        92: 'Закупуване на валута в брой',
        93: 'Закупуване на валута в брой, по сметка и за директен превод',
        94: 'Откриване на акредитив',
        95: 'Преводи чрез директен дебит и по инициатива на БНБ',
        96: 'Преводи по нареждане на титуляра с други платежни документи',
        97: 'Операции по откриване/закриване на депозити'
    },
    name='DESCRIPTION'
)

In [13]:
connection = sqlite3.connect('sebra.db')

to_sql = lambda df, table_name, index_name: df.to_sql(table_name, connection, if_exists='replace', index_label=index_name)
to_sql(df, 'payments', 'PAYMENT_ID')
to_sql(orgs, 'organizations', 'ORGANIZATION_ID')
to_sql(primary_orgs, 'primary_organizations', 'PRIMARY_ORG_CODE')
to_sql(clients, 'clients', 'CLIENT_ID')
to_sql(sebra, 'sebra_codes', 'SEBRA_PAY_CODE')

23